In [ ]:
# Import necessary libraries for environment, agents, HTTP requests, and typing
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from typing import Dict
import os
import requests
import asyncio

# Load environment variables from .env file
# override=True ensures existing variables are updated if present
load_dotenv(override=True)

In [2]:
# Define instructions for the triage agent to categorize customer queries
triage_instructions = """You are a customer support triage agent for a software company.
Your main goal is to categorize incoming customer queries into one of the following:
- 'technical_issue'
- 'billing_inquiry'
- 'feature_request'
- 'general_question'
You should also extract the key details of the customer's problem or request.
Your response should always start with the category, followed by a summary of the issue.
"""

# Create the triage agent with specified instructions and model
triage_agent = Agent(
    name="Triage Agent",
    instructions=triage_instructions,
    model="gpt-4o-mini"
)

In [ ]:
# Define a sample customer query for testing the triage agent
customer_query = "My application keeps crashing after the latest update, and I can't log in at all. I need help urgently!"

# Process the query using the triage agent and print the result
print("Triage Agent processing query:")
result = await Runner.run(triage_agent, input=customer_query)
print(result.final_output)

In [ ]:
# Define a tool to send messages to a Discord channel via a webhook
@function_tool
def send_discord_message(message_content: str, username: str = "Support Bot") -> Dict[str, str]:
    """Sends a message to a Discord channel using a webhook URL.
    Args:
        message_content (str): The content of the message to send.
        username (str): The username to display for the bot in Discord.
    Returns:
        Dict[str, str]: A dictionary indicating the status of the message sending.
    """
    webhook_url = os.environ.get('DISCORD_WEBHOOK_URL')
    if not webhook_url:
        return {"status": "error", "message": "DISCORD_WEBHOOK_URL not found in environment variables."}

    headers = {"Content-Type": "application/json"}
    payload = {
        "content": message_content,
        "username": username
    }

    try:
        response = requests.post(webhook_url, json=payload, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return {"status": "success", "code": str(response.status_code), "response": response.text}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "message": str(e)}

# Inspect the created tool to verify its configuration
print(send_discord_message)

In [ ]:
# Create an agent to send messages to a Discord channel using the send_discord_message tool
discord_reporter = Agent(
    name="Discord Reporter",
    instructions="You are an agent responsible for reporting important messages to a Discord channel. You always use the 'send_discord_message' tool.",
    tools=[send_discord_message],
    model="gpt-4o-mini"
)

# Test the Discord Reporter with a sample message
print("\nDiscord Reporter sending a message:")
message_to_send = "This is a test message from the Discord Reporter Agent."
result = await Runner.run(discord_reporter, input=f"Send the following message to Discord: {message_to_send}")
print(result.final_output)

In [ ]:
# Define instructions for the technical support agent
technical_support_instructions = """You are a highly knowledgeable technical support agent for a software company.
Your goal is to provide clear, concise, and helpful solutions for technical issues.
Always include troubleshooting steps and potential workarounds.
"""

# Define instructions for the billing support agent
billing_support_instructions = """You are a polite and efficient billing support agent.
Your goal is to address billing inquiries with accuracy and empathy.
Provide information on common billing issues, payment options, or how to contact the billing department directly.
"""

# Create specialized support agents
technical_support_agent = Agent(
    name="Technical Support Agent",
    instructions=technical_support_instructions,
    model="gpt-4o-mini"
)

billing_support_agent = Agent(
    name="Billing Support Agent",
    instructions=billing_support_instructions,
    model="gpt-4o-mini"
)

# Convert specialized agents into tools for the Support Manager
tech_tool = technical_support_agent.as_tool(
    tool_name="technical_support_response",
    tool_description="Generate a detailed technical support response for a given issue."
)

billing_tool = billing_support_agent.as_tool(
    tool_name="billing_support_response",
    tool_description="Generate a detailed billing support response for a given inquiry."
)

# Define all tools available to the Support Manager
manager_tools = [
    triage_agent.as_tool(tool_name="triage_query", tool_description="Categorize and summarize a customer query."),
    tech_tool,
    billing_tool,
    send_discord_message
]

# Define instructions for the Support Manager to orchestrate the support process
support_manager_instructions = """You are a highly capable Support Manager for a software company.
Your primary role is to process incoming customer queries.
1. First, use the 'triage_query' tool to categorize the customer's message and summarize the core issue.
2. Based on the category provided by the 'triage_query' tool:
    - If the category is 'technical_issue', use the 'technical_support_response' tool to generate a technical solution.
    - If the category is 'billing_inquiry', use the 'billing_support_response' tool to generate a billing response.
    - For 'feature_request' or 'general_question', generate a polite acknowledgment and state that the request has been noted or a general answer.
3. Once a response is generated (either by a specialized agent tool or directly by you for general queries/feature requests), use the 'send_discord_message' tool to send the final customer-facing response to the Discord channel, displaying the username as 'Customer Support'.
"""

# Create the Support Manager agent
support_manager = Agent(
    name="Support Manager",
    instructions=support_manager_instructions,
    tools=manager_tools,
    model="gpt-4o-mini"
)

# Define sample customer queries for testing the support flow
customer_query_1 = "I can't log in after the update, it says 'invalid credentials' even though I'm sure my password is correct. What should I do?"
customer_query_2 = "My last bill seems higher than usual, I'm not sure why. Can you check my subscription details?"
customer_query_3 = "I'd love to see a dark mode feature in your next update! It would be much easier on my eyes."

# Run the support flow for a technical issue
print("\n--- Running Support Flow for Technical Issue ---")
with trace("Automated Customer Support - Technical"):
    result_tech = await Runner.run(support_manager, customer_query_1)
    print(f"Final output for technical issue: {result_tech.final_output}")

# Run the support flow for a billing inquiry
print("\n--- Running Support Flow for Billing Inquiry ---")
with trace("Automated Customer Support - Billing"):
    result_billing = await Runner.run(support_manager, customer_query_2)
    print(f"Final output for billing inquiry: {result_billing.final_output}")

# Run the support flow for a feature request
print("\n--- Running Support Flow for Feature Request ---")
with trace("Automated Customer Support - Feature Request"):
    result_feature = await Runner.run(support_manager, customer_query_3)
    print(f"Final output for feature request: {result_feature.final_output}")